# Hate Speech Detector - EN - Features extraction for Conv. & LSTM model

Based on [this notebook](https://github.com/t-davidson/hate-speech-and-offensive-language/blob/master/classifier/final_classifier.ipynb).

In [1]:
import os
import pandas as pd
import csv
import numpy as np
import pickle
from klepto.archives import dir_archive
import sys
import nltk
import string
import re
import fasttext
from pymagnitude import *

from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

from tqdm.notebook import tqdm
%matplotlib inline

In [2]:
MODEL = 'lstm'

## Davidson et al. data loading

### Classes pre:
    0 - hate speech
    1 - offensive language
    2 - neither

In [3]:
if not os.path.exists('hsd/Reddit/perfect_data.pkl'):
    tweets, labels = [], []
    with open('hsd/Reddit/clean_data.csv', 'r') as f:
        for d in list(csv.reader(f))[1:]:
            tweets.append(d[1])  # tweet
            labels.append(d[2])  # class
    with open('hsd/Reddit/perfect_data.pkl', 'wb') as f:
        def chcl(c):
            return 0 if c=='2' else 1
        labels = list(map(chcl, labels))
        pickle.dump((tweets, labels), f)
else:
    with open('hsd/Reddit/perfect_data.pkl', 'rb') as f:
        tweets, labels = pickle.load(f)

### Classes post:
    0 - no hate
    1 - hate speech

In [4]:
print('Tweets: {}'.format(len(tweets)))
print('Labels: {}'.format(len(labels)))

Tweets: 79915
Labels: 79915


In [5]:
list(zip(tweets[:5], labels[:5]))

[("i joined gab to remind myself how retarded jew haters are. You wouldn't be typing on your abacus without them you retard.",
  0),
 ('This is what the left is really scared of.    https://redd.it/9rfkts', 0),
 ('That literally looks like a monkey. Why are we supposed to pretend it’s a person bc it’s wearing a red hat?',
  0),
 ('Dumb Cunt', 0),
 ('It makes you an asshole.', 0)]

## Features extraction

In [6]:
def pos_tagger(nltk_tag): 
    if nltk_tag.startswith('J'): 
        return wordnet.ADJ 
    elif nltk_tag.startswith('V'): 
        return wordnet.VERB 
    elif nltk_tag.startswith('N'): 
        return wordnet.NOUN 
    elif nltk_tag.startswith('R'): 
        return wordnet.ADV 
    else:           
        return None

def word_tokenization(tweet):
    lemmatizer = WordNetLemmatizer() 
    tokens = word_tokenize(tweet)
    words = [word for word in tokens if word.isalpha()]
    # stop_words = set(stopwords.words('english'))
    # words = [w for w in words if not w in stop_words]
    tags = nltk.pos_tag(words)
    # words = [lemmatizer.lemmatize(w[0]) if pos_tagger(w[1]) is None else lemmatizer.lemmatize(w[0], pos_tagger(w[1])) for w in tags]
    tags = [x[1] for x in tags]
    return words, tags

def preprocess(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE
    4) hashtags with HASHTAGHERE

    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    hashtag_regex = '#[\w\-]+'
    parsed_text = text_string.encode('ascii', 'ignore').decode('ascii')
    parsed_text = re.sub(space_pattern, ' ', parsed_text)
    parsed_text = re.sub(giant_url_regex, '', parsed_text)
    parsed_text = re.sub(mention_regex, '', parsed_text)
    parsed_text = parsed_text.strip('#')
    list_words, tag_list = word_tokenization(parsed_text)
    parsed_text = " ".join(list_words)
    tag_str = ' '.join(tag_list)
    return parsed_text, tag_str

def basic_tokenize(tweet):
    tweet = " ".join(re.split(" ", tweet.lower())).strip()
    return tweet.split()

# def get_pos_string(tweet):
#     text = preprocess(tweet)
#     tokens = word_tokenize(text)
#     tags = nltk.pos_tag(tokens)
#     tag_list = [x[1] for x in tags]
#     tag_str = ' '.join(tag_list)
    
    # return tag_str

def pad_words(words, length):
    if len(words) >= length:
        return words[:length]
    else:
        additional = length - len(words)
        return words + ['EMPTY']*additional

### Median sentences length

In [7]:
def median_sentences_length(data):
    all_lengths, wt_lengths, pos_lengths = [], [], []
    for d in data:
        sentence, pos_string = preprocess(d)
        # sentence = preprocess(d)
        # pos_string = get_pos_string(d)
        all_lengths.append(len(sentence.split(' ')))
        all_lengths.append(len(pos_string.split(' ')))
        wt_lengths.append(len(sentence.split(' ')))
        pos_lengths.append(len(pos_string.split(' ')))
    
    return int(np.median(all_lengths)), int(np.median(wt_lengths)), int(np.median(pos_lengths))

In [8]:
opt_length, opt_wt_length, opt_pos_length = median_sentences_length(tweets)
dim = 5*60 if MODEL == 'conv' else 200

print('Optimal all length: {}'.format(opt_length))
print('Optimal sentence length: {}'.format(opt_wt_length))
print('Optimal pos sentence length: {}'.format(opt_pos_length))

Optimal all length: 16
Optimal sentence length: 16
Optimal pos sentence length: 16


### Supervised fastText wordtokens training

In [9]:
if not os.path.exists('hsd/Reddit/fasttext.ft'):
    with open('hsd/Reddit/fasttext.ft', 'a') as f:
        for t, l in list(zip(tweets, labels)):
            text = preprocess(t)[0]
            if len(text) > 0:
                f.write('__label__{} {}\n'.format(l, text))

# load fasttext model or train & save if none
if os.path.exists('hsd/Reddit/fasttext_{}.bin'.format(MODEL)):
    ft_model = fasttext.load_model('hsd/Reddit/fasttext_{}.bin'.format(MODEL))
else:
    ft_model = fasttext.train_supervised('hsd/Reddit/fasttext.ft',
                                         lr=0.5, epoch=50, wordNgrams=3, dim=dim)
    ft_model.save_model('hsd/Reddit/fasttext_{}.bin'.format(MODEL))

### Wordtoken features

In [10]:
def get_wordtoken_fts(data, length):
    
    sentences_words = []
    for d in data:
        sentence = preprocess(d)[0]
        sentences_words.append(sentence.split(' '))
    
    sentences_words = [pad_words(sw, length) for sw in sentences_words]
    # vecs = Magnitude(MagnitudeUtils.download_model('http://magnitude.plasticity.ai/fasttext/medium/crawl-300d-2M.magnitude'))
    
    ft_matrices = []
    for sw in sentences_words:
        ft_matrix = []
        for w in sw:
            ft_matrix.append(ft_model[w])
        ft_matrices.append(ft_matrix)
    
    return ft_matrices

In [11]:
wordtoken_features = get_wordtoken_fts(tweets, opt_wt_length)
ft_model = None

In [12]:
wordtoken_features[0]

.24514595e-02, -8.40599388e-02,  2.44023297e-02,
         7.46489391e-02,  1.26412183e-01, -3.72436866e-02,  8.34057853e-02,
         2.17102878e-02,  1.12622686e-01, -3.40269729e-02,  1.01770625e-01,
         4.39640284e-02,  1.04632691e-01,  4.98048402e-03, -3.90101410e-02,
        -8.44095740e-03,  5.00477888e-02,  7.89609924e-02, -5.49028516e-02,
         8.82558152e-03, -4.81705219e-02,  1.03650272e-01, -1.06397934e-01,
        -4.92627406e-03,  1.66560084e-01,  2.92790104e-02,  1.87057478e-03,
        -7.90200382e-02, -3.97116952e-02, -1.19712904e-01, -6.38988474e-03,
         5.92313930e-02, -9.64245126e-02, -2.63133030e-02, -8.65297243e-02,
         1.34614715e-02,  6.37469515e-02,  2.22038716e-01,  1.41643703e-01,
        -1.61603019e-01, -8.85932893e-03, -1.29745649e-02,  2.77107228e-02,
        -4.24158163e-02,  6.89613372e-02, -1.99576281e-02,  3.93840261e-02,
        -5.03713004e-02,  1.24026366e-01, -6.70504272e-02, -9.07638744e-02,
         9.74184740e-03,  4.11613146e-0

### Supervised fastText pos training

In [13]:
if not os.path.exists('hsd/Reddit/fasttext_pos.ft'):
    with open('hsd/Reddit/fasttext_pos.ft', 'a') as f:
        for t, l in list(zip(tweets, labels)):
            f.write('__label__{} {}\n'.format(l, preprocess(t)[1]))

# load fasttext pos model or train & save if none
if os.path.exists('hsd/Reddit/fasttext_pos_{}.bin'.format(MODEL)):
    ft_pos_model = fasttext.load_model('hsd/Reddit/fasttext_pos_{}.bin'.format(MODEL))
else:
    ft_pos_model = fasttext.train_supervised('hsd/Reddit/fasttext_pos.ft',
                                             lr=0.5, epoch=50, wordNgrams=3, dim=dim)
    ft_pos_model.save_model('hsd/Reddit/fasttext_pos_{}.bin'.format(MODEL))

### Part of speech (PoS) features

In [14]:
def get_pos_fts(data, length):

    #Get POS tags for tweets and save as a string
    pos_sentences = []
    for d in data:
        pos_string = preprocess(d)[1]
        pos_sentences.append(pos_string)
        
        
    pos_tags = []
    for ps in pos_sentences:
        pos_tags.append(ps.split(' '))
    
    pos_tags = [pad_words(pt, length) for pt in pos_tags]
    
    ft_matrices = []
    for pt in pos_tags:
        ft_matrix = []
        for t in pt:
            ft_matrix.append(ft_pos_model[t])
        ft_matrices.append(ft_matrix)
    
    return ft_matrices

In [15]:
pos_features = get_pos_fts(tweets, opt_pos_length)
tweets = None
ft_pos_model = None

In [16]:
pos_features[0]

1.23508750e-02,  5.72165549e-02,
        -1.31145746e-01, -1.24881476e-01, -9.48407501e-02, -2.25890264e-01,
         7.91670755e-02, -1.49243191e-01, -6.45336285e-02, -9.84818786e-02,
        -5.22555448e-02,  1.57493632e-02,  3.40449065e-01, -1.64926387e-05,
        -9.17234048e-02,  3.25061530e-02, -9.13664773e-02,  1.58823375e-02,
        -1.27016097e-01, -6.25679716e-02, -3.26130819e-03,  4.69549328e-01,
        -1.49927959e-01,  5.84794506e-02,  4.00965065e-02, -4.11393717e-02,
         3.74275725e-04,  1.75462633e-01, -1.97319090e-01,  8.93582180e-02,
        -1.82304412e-01,  1.41307473e-01,  1.74809039e-01,  2.09229782e-01,
         1.35128215e-01, -1.66464612e-01, -1.44204825e-01, -1.81926206e-01,
        -1.41884061e-02,  2.33787112e-02,  6.26844317e-02, -9.98031273e-02,
         8.14773981e-03,  1.26820663e-02,  3.56209278e-01, -1.12697989e-01,
        -3.18144299e-02,  1.50632933e-01, -1.12017803e-01, -3.32333595e-02,
        -1.51067942e-01,  3.20639019e-03, -7.86744244e-

In [17]:
np.array(wordtoken_features)

array([[[ 1.21994009e-02, -3.50707695e-02,  8.26576054e-02, ...,
         -5.60431294e-02,  2.40664892e-02,  9.95721295e-02],
        [ 5.86541928e-03, -1.89711642e-03,  2.94873510e-02, ...,
         -2.39114463e-02,  2.48002913e-03,  2.46266779e-02],
        [-7.60615692e-02,  4.71917428e-02, -3.18519354e-01, ...,
          2.17330664e-01, -5.46958745e-02, -2.85313904e-01],
        ...,
        [ 2.58562411e-03, -2.40624081e-02,  3.93422619e-02, ...,
         -3.57299112e-03,  3.43654375e-03,  4.07229997e-02],
        [-1.22412089e-02,  8.71368032e-03, -3.95835713e-02, ...,
          2.61587594e-02, -4.98380186e-03, -2.79369410e-02],
        [-1.67658217e-02,  9.62103065e-03, -6.24215342e-02, ...,
          7.10478052e-02, -1.87845677e-02, -5.25574200e-02]],

       [[ 4.81294096e-02,  7.65520781e-02, -1.11260653e-01, ...,
          6.62405342e-02,  4.89832573e-02, -2.66722381e-01],
        [-3.78670916e-02,  2.99157538e-02, -1.58763468e-01, ...,
          5.92896268e-02, -3.39349546e

In [18]:
words_token_shape = np.array(wordtoken_features).shape[1]

In [19]:
np.array(pos_features).shape

(79915, 16, 200)

### All features

In [20]:
# #Now join them all up
# features = np.array(wordtoken_features)
features = np.concatenate([wordtoken_features, pos_features], axis=1)
wordtoken_features = None
pos_features = None

In [21]:
features.shape

(79915, 32, 200)

In [22]:
features[0]


array([[ 0.0121994 , -0.03507077,  0.08265761, ..., -0.05604313,
         0.02406649,  0.09957213],
       [ 0.00586542, -0.00189712,  0.02948735, ..., -0.02391145,
         0.00248003,  0.02462668],
       [-0.07606157,  0.04719174, -0.31851935, ...,  0.21733066,
        -0.05469587, -0.2853139 ],
       ...,
       [-0.21206434,  0.2475656 , -0.1156081 , ...,  0.18993613,
         0.04229975,  0.57917786],
       [ 0.10508851,  0.03171046,  0.48890537, ..., -0.09775457,
        -0.27208284, -0.01912209],
       [-0.5175854 ,  0.3176709 , -0.03745794, ..., -0.1350805 ,
         0.09059026, -0.44849277]], dtype=float32)

## Save features & labels

In [23]:
archive = dir_archive('hsd/Reddit/X_y_{}'.format(MODEL), {'features': features, 'labels': labels,
                                                                'wt_num': words_token_shape}, serialized=True)
archive.dump()
del archive